<a href="https://colab.research.google.com/github/HandanYU/Rumour-detection/blob/main/Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- replace_abbreviations
- remove url 
- get the emoji and replace them as words
- remove stop words
- keep only english letters
- stemming

In [ ]:
# !pip install emoji
# !pip install nltk
# !pip install textblob

  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=5a92f91dcf134fc24011694b0e97f509edb85899e2f6e4aeb666514621a3bb94
  Stored in directory: c:\users\trist\appdata\local\pip\cache\wheels\5e\8c\80\c3646df8201ba6f5070297fe3779a4b70265d0bfd961c15302
Successfully built emoji


### Sort raw data by time

In [35]:
@timer('ms')

def sort_by_time(raw_file, json_file):
    with open(raw_file) as file:
        ids = file.readlines()

    with open(json_file, 'r+') as file:
        content = file.read()
        content=json.loads(content)
        df = pd.DataFrame(content)
        df = df.T

    save_name = raw_file[:-4] + '_sorted.txt'
    with open(save_name, 'w') as file:
        date = pd.Series(pd.DatetimeIndex(df.iloc[:, 6]), index=df.index)
        df.drop(['created_at'], axis=1, inplace=True)
        df['time'] = date

        for id_ in ids:
            ids_ = id_.strip().split(',')
            source_id = ids_[0]
            file.write(source_id)
            if len(ids_) > 1:
                reply_ids = ids_[1:]
                reply_ids[-1] = reply_ids[-1].replace('\n', '')
                valid_ids = [index for index in reply_ids if index in df.index]
                sorted_replies = df.loc[valid_ids].sort_values(by='time')
                if len(valid_ids) > 0:
                    file.write(',')

                for i, index in enumerate(sorted_replies.index):
                    file.write(index)
                    if i != len(sorted_replies.index) - 1:
                        file.write(',')

            file.write('\n')

In [ ]:
raw_files = ['train.data.txt', 'dev.data.txt']
json_files = ['./filtered data/train_reply.json', './filtered data/dev_reply.json']
for raw_file, json_file in zip(raw_files, json_files):
    sort_by_time(raw_file, json_file)

In [ ]:
with open(raw_files[0]) as file:
    train = file.readlines()
with open('train.data_sorted.txt') as file:
    s = file.readlines()
assert len(train) == len(s)

## Text pre-processing

In [18]:
import emoji
from nltk.corpus import stopwords
import re
import nltk
import json
import pandas as pd
from utils import timer
from datetime import datetime
from textblob import TextBlob
import numpy as np
# from nltk.corpus import wordnet

In [19]:
nltk.download('wordnet')
stemmer = nltk.stem.porter.PorterStemmer()
stopword = stopwords.words('english')

[nltk_data] Error loading wordnet: <urlopen error [Errno 61]
[nltk_data]     Connection refused>


In [27]:



# @timer('ms')
# def replace_duplicate_letter(word):
#     repeat_reg = re.compile(r'(\w*)(\w)\2(\w*)')
#     repl = r'\1\2\3'
#     if wordnet.synsets(word):  # 判断当前字符串是否是单词
#         return word
#     repl_word = repeat_reg.sub(repl, word)
#     if repl_word != word:
#         return replace_duplicate_letter(repl_word)
#     else:
#         return repl_word
  
def clean_tweet(content):
    from time import strftime
    # def compute_num_month(content):
    #     month_num = 0
    #     month = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "octorber", "november", "december", 
    #             "jan.", "feb.", "mar.", "apr.", "may.", "jun.", "jul.", "aug.", "sept.", "oct.", "nov.", "dec."]
    #     for i in content:
    #         if i in month:
    #             month_num += 1
    #     return month_num
    # replace_abbreviations
    content = content.lower()
    content = re.sub(r"won't", "will not", content)
    content = re.sub(r"can't", "can not", content)
    content = re.sub(r"cannot", "can not", content)
    content = re.sub(r"n't", " not", content)
    content = re.sub(r"'re", " are", content)
    content = re.sub(r"'s", " is", content)
    content = re.sub(r"'d", " would", content)
    content = re.sub(r"'ll", " will", content)
    content = re.sub(r"'t", " not", content)
    content = re.sub(r"'ve", " have", content)
    content = re.sub(r"'m", " am", content)
    content = re.sub(r".”", " ", content)
    
    # get the number of month be mentioned
    # month_num = compute_num_month(content)

    # get the number of url
    mentioned_url_num = len(re.findall(r'https?://[^ ]+', content))
    mentioned_url_num += len(re.findall(r'www.[^ ]+', content))
    # get the number of twitter ID be mentioned
    id_num = len(re.findall(r'@[A-Za-z0-9_]+', content))
    content = re.sub(r'@[A-Za-z0-9_]+', '', content) # remove twitter ID
    # remove url 
    ## http, https
    content = re.sub(r'https?://[^ ]+', '', content) 
    ## www.
    content = re.sub(r'www.[^ ]+', '', content)
    # get the emoji and replace them as words
    emojis = emoji.distinct_emoji_list(content)
    for e in emojis:
        content = re.sub(e, emoji.demojize(e), content)
    content = re.sub('\w+\d+\w+', '', content) # remove the word contains numbers
    
    content = re.sub(r'[:_!+“\-=——,$%^\?\\~\"\'@#$%&*<>{}\[\]()/]', ' ', content) # remove punctuation, except .
    
    content = re.sub(r"\s+", " ", content) # conver multiple spaces as a single space
    content = content.strip()
    
    # remove stop words 
    # TODO: and keep only english letters
    
    content = [c for c in content.split(' ') if c not in stopword and c.isalpha()]
    # do stemming
    content = [stemmer.stem(token.strip()) for token in content]
    
    return ' '.join(content), mentioned_url_num, id_num 
    #, month_num

In [28]:
# @timer('ms')
def json2df(json_file):
    """
    json_file: 'train_reply.json'
    """
    with open(json_file,'r+') as file:
        content = file.read()
    content = json.loads(content)
    df = pd.DataFrame(content)
    df = df.T
    return df
# df = json2df('train_reply.json')
def clean_data(data_type):
  """
  Args: 
    data_type: 'dev', 'train'
  Returns:
    source_df, reply_df
  """
  source_df = json2df(f'./data/full data/{data_type}_source.json')
  source_df['temp'] = source_df['text'].apply(lambda x: clean_tweet(x))
  source_df['text'] = source_df['temp'].apply(lambda x: x[0])
  source_df['mentioned_url_num'] = source_df['temp'].apply(lambda x: x[1])
  source_df['id_num'] = source_df['temp'].apply(lambda x: x[2])
  source_df = source_df.drop(columns='temp')
  source_df['tweet_id'] = source_df.index

  reply_df = json2df(f'./data/full data/{data_type}_reply.json')
  reply_df['temp'] = reply_df['text'].apply(lambda x: clean_tweet(x))
  reply_df['text'] = reply_df['temp'].apply(lambda x: x[0])
  reply_df['mentioned_url_num'] = reply_df['temp'].apply(lambda x: x[1])
  reply_df['id_num'] = reply_df['temp'].apply(lambda x: x[2])
  reply_df = reply_df.drop(columns='temp')
  reply_df['tweet_id'] = reply_df.index
  
  return source_df, reply_df
# train_source_df, train_reply_df = clean_data('train')

# dev_source_df, dev_reply_df = clean_data('dev')


## Concat label

In [29]:
import pandas as pd
def concat_label(data_type, source_feature_df):
    """Concat labels on source tweets
    data_type: 'dev', 'train'
    """
    df = pd.DataFrame(columns=['tweet_id', 'label'])
    with open(f'./data/original_data/{data_type}_source.txt', 'r') as f:
        ids = f.readlines()
    with open(f'./data/original_data/{data_type}.label.txt', 'r') as f:
        labels = f.readlines()
    df['tweet_id'] = [id.strip() for id in ids]
    df['label'] = [label.strip() for label in labels]
    df_labels = pd.merge(source_feature_df, df, on='tweet_id', how='left')
    df_labels['label'] = df_labels['label'].apply(lambda x: 0 if x == 'nonrumour' else 1)
    return df_labels

# dev_source_df = concat_label('dev', dev_source_df)
# train_source_df = concat_label('train', train_source_df)


## Concat Sorted reply ids

In [30]:
def concat_reply(data_type, source_df):
    """concat replies on source tweets
    data_type: 'dev', 'train'
    """
    df = pd.DataFrame(columns=['tweet_id', 'reply'])
    with open(f'./data/original_data/{data_type}.data_sorted.txt', 'r') as f:
        content = f.readlines()
    df['tweet_id'] = [c.split(',')[0].strip() for c in content]
    df['reply'] = [','.join([i.strip() for i in c.split(',')[1:]]) for c in content]
    source_df = pd.merge(source_df, df, on='tweet_id', how='left')
    return source_df

# dev_source_df = concat_reply('dev', dev_source_df)
# train_source_df = concat_reply('train', train_source_df)


## check whether the created date is on weekday

In [31]:
def check_weekday(df):
    """
    df: source_df or reply_df
    """
    df['isoweekday'] = df['created_at'].apply(lambda x: datetime.strptime(x.split('T')[0], '%Y-%m-%d').isoweekday())
    df['isweekday'] = df['isoweekday'].apply(lambda x: 1 <= x <= 5)
    df = df.drop(columns='isoweekday')
    return df

## Concat user info

In [32]:
def concat_user_info(data_type, source_df):
    """concat user info on source tweets
    data_type: 'dev', 'train'
    """
    df = json2df(f'./data/full data/{data_type}_source_userinfo.json')
    df['user_id'] = df.index
    source_df = pd.merge(source_df, df, on='user_id', how='left')
    return source_df



## Concat reply's features

In [33]:
def concat_reply_info(reply_ids, reply_df, statis_feature):
    statistic_dict = dict()   
    replies_txt = ''
    if reply_ids == '':
        return [np.nan] * (len(statis_feature) + 1)
    for r in reply_ids.split(','):
        # reply_num = len(reply_ids.split(','))
        cur_reply_txt = reply_df.loc[r]['text'].strip()
        if cur_reply_txt != '':
            replies_txt += cur_reply_txt + ' [SEP] ' ## TODO
        ## get statistic features
        for f in statis_feature:
            statistic_dict[f] = statistic_dict.get(f, 0) + int(reply_df.loc[r][f])
    res_values = [replies_txt.strip()]
    for f in statis_feature:
        res_values.append(statistic_dict[f])
    return res_values

# Processing Text features and get statistic features

In [34]:
def processing(data_type):
    source_df, reply_df = clean_data(data_type)
    # source_df: 
    # ['text', 'reply_count', 'like_count', 'retweet_count', 'quote_count',
    # 'possibly_sensitive', 'created_at', 'user_id', 'has_url',
    # 'mentioned_url_num', 'id_num', 'tweet_id']

    # reply_df:
    # ['text', 'reply_count', 'like_count', 'retweet_count', 'quote_count',
    # 'possibly_sensitive', 'created_at', 'user_id', 'has_url',
    # 'mentioned_url_num', 'id_num', 'tweet_id']
    
    source_df = concat_label(data_type, source_df)
    source_df = concat_reply(data_type, source_df)
    source_df = check_weekday(source_df)
    source_df = concat_user_info(data_type, source_df)
    reply_df = check_weekday(reply_df)
    # add sentiment score
    source_df['senti_score'] = source_df['text'].apply(lambda x: 1 if TextBlob(x).sentiment.polarity > 0 else 0)
    reply_df['senti_score'] = reply_df['text'].apply(lambda x: 1 if TextBlob(x).sentiment.polarity > 0 else 0)
    reply_df.index = reply_df['tweet_id']
    source_df.index = source_df['tweet_id']
    # concat replies info to source_df
    statis_feature = ['reply_count', 'like_count', 'retweet_count', 'quote_count', 
                        'possibly_sensitive', 'has_url', 'mentioned_url_num', 
                        'id_num', 'isweekday', 'senti_score']
    source_df[['reply_text'] + ['reply_' + s for s in statis_feature]] = source_df.apply(lambda x: concat_reply_info(x['reply'], reply_df, statis_feature), axis=1, result_type='expand')                
    source_df = source_df.loc[~source_df['reply_text'].isnull()]
    return source_df

In [35]:
train_df = processing('train')
dev_df = processing('dev')

## Process Statistic Features

In [36]:
def extract_stat_tweet_feat(df):
    # extract statistic features
    statistic_features = ['reply_reply_count', 'reply_like_count',
       'reply_retweet_count', 'reply_quote_count', 'reply_possibly_sensitive',
       'reply_has_url', 'reply_mentioned_url_num', 'reply_id_num',
       'reply_isweekday', 'reply_senti_score', 'reply_count', 'like_count', 'retweet_count', 'quote_count',
       'possibly_sensitive', 'has_url',
       'mentioned_url_num', 'id_num', 'isweekday', 'followers_count', 'tweet_count', 'verified',
       'senti_score' ]
    stat_feat_df = df[statistic_features]
    stat_feat_df.index = df['tweet_id']
    tweet_df = df.drop(columns=statistic_features)
    tweet_df.index = df['tweet_id']
    # convert into float
    for col in ['tweet_count', 'followers_count', 'verified']:
        stat_feat_df[col] = stat_feat_df[col].apply(lambda x: float(x))
    # fill nan using corresponding mean
    stat_feat_df = stat_feat_df.fillna(stat_feat_df.mean())
    return stat_feat_df, tweet_df
                

In [37]:
train_stat_feat_df, train_tweet_df = extract_stat_tweet_feat(train_df)
dev_stat_feat_df, dev_tweet_df = extract_stat_tweet_feat(dev_df)

/Users/yuhandan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [46]:
stat_feat_df = pd.read_csv('./data/filtered data/test_stat_feat_df.csv')

## Min-Max Scaler

In [47]:
stat_feat_df.index = stat_feat_df['tweet_id']
stat_feat_df = stat_feat_df[train_stat_feat_df.columns]



In [48]:
from sklearn import preprocessing
minmax = preprocessing.MinMaxScaler()
train_scaled_stat_feat_df = pd.DataFrame(columns=train_stat_feat_df.columns, index=train_stat_feat_df.index,
                                         data=minmax.fit_transform(train_stat_feat_df))
test_scaled_stat_feat_df = pd.DataFrame(columns=stat_feat_df.columns, index=stat_feat_df.index,
                                       data=minmax.transform(stat_feat_df))


In [50]:
test_scaled_stat_feat_df.to_csv('./data/filtered data/test_stat_feat_df.csv')

In [3]:
from sklearn import preprocessing
minmax = preprocessing.MinMaxScaler()
train_scaled_stat_feat_df = pd.DataFrame(columns=train_stat_feat_df.columns, index=train_stat_feat_df.index,
                                         data=minmax.fit_transform(train_stat_feat_df))
dev_scaled_stat_feat_df = pd.DataFrame(columns=dev_stat_feat_df.columns, index=dev_stat_feat_df.index,
                                       data=minmax.transform(dev_stat_feat_df))


NameError: name 'pd' is not defined

In [2]:
dev_tweet_df.to_csv('./data/filtered data/dev_tweet_df.csv')
dev_scaled_stat_feat_df.to_csv('./data/filtered data/dev_stat_feat_df.csv')
train_tweet_df.to_csv('./data/filtered data/train_tweet_df.csv')
train_scaled_stat_feat_df.to_csv('./data/filtered data/train_stat_feat_df.csv')

NameError: name 'dev_tweet_df' is not defined

In [4]:
import pandas as pd
dev_scaled_stat_feat_df = pd.read_csv('./data/filtered data/dev_stat_feat_df.csv')

/Users/yuhandan/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [5]:
dev_scaled_stat_feat_df.columns

Index(['tweet_id', 'reply_reply_count', 'reply_like_count',
       'reply_retweet_count', 'reply_quote_count', 'reply_possibly_sensitive',
       'reply_has_url', 'reply_mentioned_url_num', 'reply_id_num',
       'reply_isweekday', 'reply_senti_score', 'reply_count', 'like_count',
       'retweet_count', 'quote_count', 'possibly_sensitive', 'has_url',
       'mentioned_url_num', 'id_num', 'isweekday', 'followers_count',
       'tweet_count', 'verified', 'senti_score'],
      dtype='object')